**About** : This notebook is used to train models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
#     init_neptune,
)

from params import *
from data.dataset import *
from data.preparation import *
from util.metrics import rsna_loss
from util.plots import plot_confusion_matrix
from model_zoo.models_lvl2 import define_model
from inference.extract_features import Config
from training.main_lvl2 import k_fold, retrieve_preds

## Data

In [ ]:
df_patient, df_img = prepare_data(DATA_PATH)

In [ ]:
EXP_FOLDERS = [
#         ("../logs/2023-09-06/4/", "seg"),  # v2-s
        ("../logs/2023-09-19/10/", "seg"),  # v2-rw-t stride+
        ("../logs/2023-09-24/20/", "seg3d"),
#         ("../logs/2023-09-18/79/", "probas"),  # b5
#         ("../logs/2023-09-18/94/", "probas"),  # v2m
#         ("../logs/2023-09-19/7/", "probas"),  # v2s
        ("../logs/2023-09-18/90/", "probas"),  # convnext-tiny
    ]
EXP_FOLDER = EXP_FOLDERS[0][0]

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [ ]:
if "fold" not in df_patient.columns:
    folds = pd.read_csv(config.folds_file)
    df_img = df_img.merge(folds)
    df_patient = df_patient.merge(folds)

In [ ]:
dataset = PatientFeatureDataset(
    df_patient[df_patient['fold'] == 0],
    df_img[df_img['fold'] == 0],
    EXP_FOLDERS,
    max_len=200,
    resize=None
)

In [ ]:
ft, y, y_aux = dataset[0]

In [ ]:
# lens = []
# for i in tqdm(range(len(dataset))):
#     x = dataset[i][0]
#     lens.append(len(x))
# #     break

# sns.histplot(lens)
# plt.show()

In [ ]:
lens = []
for i in tqdm(range(len(dataset))):
    fts = dataset[i][0]
    x = fts.numpy()
    lens.append(len(x))
    
#     start, end = detect_start_end(x)
    
    if len(x) > 1000:
#         plt.subplot(1, 2, 1)
        plt.plot(x[:, :5])
#         plt.axvline(start, c="salmon")
#         plt.axvline(end, c="salmon")
        
#         plt.subplot(1, 2, 2)
#         plt.plot(kept)
        plt.show()
    
    break

In [ ]:
# sns.histplot(lens)

## Model

In [ ]:
model = define_model("transfo", ft_dim=x.shape[-1], layer_dim=512, n_layers=1, dense_dim=256, num_classes=11, num_classes_aux=0)

In [ ]:
model = model.cuda()

In [ ]:
pred, pred_aux = model(fts.unsqueeze(0).cuda())
pred.size()

## Training
- Handle variable sequence length more cleverly
- Tweak CNN
- Tweak fancier archs

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    exp_folders = [
#         ("../logs/2023-09-06/4/", "seg"),  # v2-s
#         ("../logs/2023-09-19/10/", "seg"),  # v2-rw-t stride+
        ("../logs/2023-09-20/14/", "seg"),  # v2-rw-t stride+ 384
#         ("../logs/2023-09-25/26/", "seg"),  # v2-rw-t stride+ 384 4 classes
#         ("../logs/2023-09-24/20/", "seg3d"),  # resnet18d 3D

        ("../logs/2023-09-20/36/", "probas"),  # convnext-tiny
#         ("../logs/2023-09-22/9/", "probas"),  # v2m
#         ("../logs/2023-09-21/32/", "probas"),  # v2m
#         ("../logs/2023-09-22/24/", "probas"),  # convnext-nano
    ]

    restrict = True
    max_len = 600 if restrict else 1000
    resize = 200
    n_fts = 0  # already pooled features, not supported yet

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
#     name = "rnn"
    name = "transfo"
    ft_dim = (11 + 11) * len([p for p in exp_folders if "probas" in p[1]]) + 5 # + 4

    dense_dim = 256  # 384
    layer_dim = 128
    n_layers = 1

    p = 0.1
    use_msd = False
    num_classes = 11
    num_classes_aux = 0

    # Training    
    loss_config = {
        "name": "patient",
        "weighted": True,
        "use_any": True,
        "smoothing": 0,
        "activation": "patient",
        "aux_loss_weight": 0,
        "name_aux": "patient",
        "smoothing_aux": 0,
        "activation_aux": "",
    }

    data_config = {
        "batch_size": 64,
        "val_bs": 256,
        "mix": "mixup",
        "mix_proba": 0.,
        "sched": False,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 5e-4,  # 7e-4, 9e-4
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
        "weight_decay": 0.2,
    }

    epochs = 8

    use_fp16 = True
    verbose = 1
    verbose_eval = 50

    fullfit = False
    n_fullfit = 1

    local_rank = 0
    distributed = False
    world_size = 1

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    exp_folders = [
#         ("../logs/2023-09-06/4/", "seg"),  # v2-s
#         ("../logs/2023-09-19/10/", "seg"),  # v2-rw-t stride+
        ("../logs/2023-09-20/14/", "seg"),  # v2-rw-t stride+ 384
#         ("../logs/2023-09-25/26/", "seg"),  # v2-rw-t stride+ 384 4 classes
#         ("../logs/2023-09-24/20/", "seg3d"),  # resnet18d 3D

        ("../logs/2023-09-20/36/", "probas"),  # convnext-tiny
#         ("../logs/2023-09-22/9/", "probas"),  # v2m
#         ("../logs/2023-09-21/32/", "probas"),  # v2m
#         ("../logs/2023-09-22/24/", "probas"),  # convnext-nano
#         ("../logs/2023-09-25/22/", "probas"),  # convnext-tiny 
#         ("../logs/2023-09-25/15/", "probas"),  # convnext-tiny 
    ]

    restrict = True
    max_len = 600 if restrict else 1000
    resize = 200
    n_fts = 0  # already pooled features, not supported yet

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
#     name = "rnn"
    name = "rnn_att"
    ft_dim = (11 + 11) * len([p for p in exp_folders if "probas" in p[1]]) + 5 # + 4

    dense_dim = 384  # 384
    layer_dim = 256
    n_layers = 1
    
    p = 0.
    use_msd = False
    num_classes = 11
    num_classes_aux = 0

    # Training    
    loss_config = {
        "name": "patient",
        "weighted": True,
        "use_any": True,
        "smoothing": 0,
        "activation": "patient",
        "aux_loss_weight": 0,
        "name_aux": "patient",
        "smoothing_aux": 0,
        "activation_aux": "",
    }

    data_config = {
        "batch_size": 64,
        "val_bs": 256,
        "mix": "mixup",
        "mix_proba": 0.,
        "sched": False,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 5e-4,  # 7e-4, 9e-4
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
        "weight_decay": 0.2,
    }

    epochs = 8

    use_fp16 = True
    verbose = 1
    verbose_eval = 50

    fullfit = False
    n_fullfit = 1

    local_rank = 0
    distributed = False
    world_size = 1

In [ ]:
DEBUG = False
log_folder = None

In [ ]:
# if not DEBUG:
#     log_folder = prepare_log_folder(LOG_PATH)
#     print(f"Logging results to {log_folder}")
#     config_df = save_config(Config, log_folder + "config.json")
#     create_logger(directory=log_folder, name="logs.txt")

# preds, preds_aux = k_fold(Config, df_patient, df_img, log_folder=log_folder, run=None)

### Eval

In [ ]:
# !cat ../logs/2023-09-25/17/logs.txt
# !cat ../logs/2023-09-25/17/config.json

In [ ]:
# !cat ../logs/2023-09-21/27/logs.txt
# !cat ../logs/2023-09-21/27/config.json

In [ ]:
EXP_FOLDERS = [
#     "../logs/2023-09-18/82/",  # 0.394 - b5
#     "../logs/2023-09-19/0/",  # 0.388 - v2m
#     "../logs/2023-09-19/12/",  # 0.380 - v2s
#     "../logs/2023-09-19/11/",  # 0.369 - convnext-tiny
#     "../logs/2023-09-19/19/",  # 0.370 - convnext-tiny rest
#     "../logs/2023-09-19/27/",  # 0.370 - convnext-tiny rest
#     "../logs/2023-09-20/31/",  # 0.369 - convnext-tiny 384
#     "../logs/2023-09-20/28/",  # 0.370 - convnext-tiny 384 x seg 384
#     "../logs/2023-09-21/0/",  # 0.366 - convnext-tiny 384 x seg 384
#     "../logs/2023-09-21/24/",  # 0.363 - convnext-tiny 384 x seg 384 rnn_att
#     "../logs/2023-09-21/25/",  # 0.360 - convnext-tiny 384 x seg 384 rnn_att
    "../logs/2023-09-21/27/",  # 0.358 - convnext-tiny 384 x seg 384 rnn_att resize      <---
#     "../logs/2023-09-21/28/",  # 0.358 - convnext-tiny 384 x seg 384 rnn_att resize
#     "../logs/2023-09-21/31/",  # 0.361 - convnext-tiny 384 x seg 384 cnn_att resize
#     "../logs/2023-09-22/20/",  # 0.360 - convnext-tiny mixup 384 x seg 384 rnn_att 
    "../logs/2023-09-25/28/",  # 0.361 - convnext-tiny 384 x seg 384 transfo resize       <---
#     "../logs/2023-09-25/30/",  # 0.360 - convnext-tiny 384 x seg 384 rnn_att resize maxlen
#     "../logs/2023-09-25/32/",  # 0.358 - convnext-tiny 384 x seg 384 rnn_att resize seg 4 classes
#     "../logs/2023-09-25/36/",   # 0.361 - convnext-tiny 384 x seg 384 transfo resize + 3d
#     "../logs/2023-09-25/17/",   # 0.357 - convnext-tiny 384 x seg 384 rnn_att resize + 3d      <---
]

In [ ]:
preds_oof = []
for exp_folder in tqdm(EXP_FOLDERS):
    df_oof, pred_oof = retrieve_preds(df_patient, df_img, Config, exp_folder)
    preds_oof.append(pred_oof)

pred_oof = np.mean(preds_oof, 0)

In [ ]:
losses, avg_loss = rsna_loss(pred_oof, df_oof)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")
    
print(f'\n -> CV Score : {avg_loss :.3f}')

In [ ]:
PATIENT_TARGETS[0]

In [ ]:
plt.figure(figsize=(22, 4))

plt.subplot(1, 5, 1)
plot_confusion_matrix(pred_oof[:, 0] > 0.5, df_oof[PATIENT_TARGETS[0]], display_labels=["ok", "injury"], normalize=None, show_label=True)
plt.title(PATIENT_TARGETS[0])

plt.subplot(1, 5, 2)
plot_confusion_matrix(pred_oof[:, 1] > 0.5, df_oof[PATIENT_TARGETS[1]], display_labels=["ok", "injury"], normalize=None)
plt.title(PATIENT_TARGETS[1])

plt.subplot(1, 5, 3)
plot_confusion_matrix(pred_oof[:, 2:5].argmax(-1), df_oof[PATIENT_TARGETS[2]], display_labels=["ok", "low", "high"], normalize=None)
plt.title(PATIENT_TARGETS[2])

plt.subplot(1, 5, 4)
plot_confusion_matrix(pred_oof[:, 5:8].argmax(-1), df_oof[PATIENT_TARGETS[3]], display_labels=["ok", "low", "high"], normalize=None)
plt.title(PATIENT_TARGETS[3])

plt.subplot(1, 5, 5)
plot_confusion_matrix(pred_oof[:, 9:].argmax(-1), df_oof[PATIENT_TARGETS[4]], display_labels=["ok", "low", "high"], normalize=None)
plt.title(PATIENT_TARGETS[4])

plt.show()

In [ ]:
# max_frame = df_img.groupby('series').max()[['frame']].reset_index().rename(columns={"frame": "max_frame"})
# df_img = df_img.merge(max_frame, on="series")
# df_img["frame_r"] = df_img["max_frame"] - df_img["frame"]

# df_img["frame_pct"] = df_img["frame"] / df_img["max_frame"]

In [ ]:
# # dfe = df_img[df_img['bowel_injury'] == 1]
# # dfe = df_img[df_img['pred_liver'] >= 0.99]
# # dfe = df_img[df_img['pred_spleen'] >= 0.99]
# # dfe = df_img[df_img['pred_kidney'] >= 0.99]

# l = 600
# dfe = dfe[dfe['max_frame'] > l]

# plt.title((dfe['frame_pct'] < 0.4).mean())
# sns.histplot(dfe['frame_pct'])
# plt.show()

In [ ]:
# pred_oof_ = pred_oof.copy()
# losses, avg_loss = rsna_loss(pred_oof, df_oof)
# best_score = avg_loss

# for _ in range(2):
#     factors = []
#     for i in range(pred_oof.shape[1]):
#         scores = {}
#         for factor in np.round(np.arange(0.5, 1.5, 0.1), 2):
#             for shift in np.round(np.arange(-0.1, 0.11, 0.1), 2):
# #             for shift in [-0.1, 0, 0.1]:
#                 pred_oof_r = pred_oof_.copy()
#                 pred_oof_r[:, i] = pred_oof_r[:, i] * factor + shift
#                 pred_oof_r[:, i] = np.clip(pred_oof_r[:, i], 0.00001, 0.99999)

#                 losses, avg_loss = rsna_loss(pred_oof_r, df_oof)
#                 scores[(factor, shift)] = avg_loss

#     #     print(scores)
#         best_coefs, best_loss = min(scores.items(), key=operator.itemgetter(1))
#         pred_oof_[:, i] = np.clip(pred_oof_[:, i] * best_coefs[0] + best_coefs[1], 0.00001, 0.99999)
#         best_score = best_loss
#         print(f'{i} - {best_coefs}  -  {best_loss :.3f}')
#         factors.append(best_coefs)

In [ ]:
dummy = np.array(
    [
        [0.04] * len(df_oof),
        [0.3] * len(df_oof),
        [0.6] * len(df_oof), [0.05] * len(df_oof), [0.05] * len(df_oof),
        [0.4] * len(df_oof), [0.07] * len(df_oof), [0.03] * len(df_oof),
        [0.3] * len(df_oof), [0.04] * len(df_oof), [0.07] * len(df_oof),
    ]
).T
losses, avg_loss = rsna_loss(dummy, df_oof)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")
    
print(f'\n -> CV Score : {avg_loss :.3f}')

In [ ]:
# for i in range(2):
#     sns.histplot(preds[:, i])
    
#     auc = roc_auc_score(df_val[PATIENT_TARGETS[i]], preds[:, i])
#     print(f'- {PATIENT_TARGETS[i]} auc : {auc:.3f}')
    
#     plt.show()

Done ! 